In [1]:
import numpy as np
import pandas as pd
import math as ms
from itertools import repeat

In [3]:
#INPUT DESIGN VARIABLES HERE
Girder_Span=30 #Girder Span [ft], range: (10ft, 50ft)
Infill_Span=30 #Infill Span [ft], range: (10ft, 50ft)
NumInfill=2 #Number of Equally Spaced Infills, range: (1, 5)
Conc_Type_num=1 #Concrete Type, range: (0: Normal Weight Concrete, 1: Light Weight Concrete)
Deck_Type=1.5 #Deck Depth [in], range: (1.5, 2, 3)
percent_comp=75 #Percent Composite Connection: range: (25, 80)

#parametets
Es = 29000.0 #[ksi]
Fy = 50.0 #[ksi]
f_prime_c = 4 #[ksi]
Clear_Spans = 3
Building_Use_num = 3

def sizer_and_calc(Girder_Span, Infill_Span, NumInfill, Conc_Type_num, Deck_Type, Building_Use_num, percent_comp):


    def mitigation_method(Additional_Topping):
        infill_spacing = Girder_Span / (NumInfill + 1)
        infill_spacing_rounded = ms.ceil(Girder_Span / (NumInfill + 1))

        #--------------------------------------------------------------------
        #----------------------Database Imports------------------------------
        #-------------------------------------------------------------------- 

        #Read in databases
        AISC_database = pd.read_csv('AISC STEEL MANUAL V16 (Modified) Sorted.csv', encoding='latin1')
        Deck_database = pd.read_csv('DeckData.csv')

        #Select only the W-Shapes
        W_Shapes = AISC_database[(AISC_database['Type'] == 'W')] #& (AISC_database['Economic'] == 1)]

        #Deck Data Pulled from 'DeckData.xlsx'
        Deck_Type_Table = Deck_database['Deck Type']
        Conc_Type_Table = Deck_database['Conc Type']
        Total_Depth = Deck_database['Total']
        Topping_Depth = Deck_database['Topping']
        Deck_Gage = Deck_database['Deck Gage']
        Clear_Span_1 = Deck_database['1Span']
        Clear_Span_2 = Deck_database['2Span']
        Clear_Span_3 = Deck_database['3Span']
        Slab_Weight = Deck_database['Concrete + Deck (psf)']
        Allow_Load_4ft = Deck_database['4ft']
        Allow_Load_5ft = Deck_database['5ft']
        Allow_Load_6ft = Deck_database['6ft']
        Allow_Load_7ft = Deck_database['7ft']
        Allow_Load_8ft = Deck_database['8ft']
        Allow_Load_9ft = Deck_database['9ft']
        Allow_Load_10ft = Deck_database['10ft']
        Allow_Load_11ft = Deck_database['11ft']
        Allow_Load_12ft = Deck_database['12ft']
        Allow_Load_13ft = Deck_database['13ft']
        Allow_Load_14ft = Deck_database['14ft']
        Allow_Load_15ft = Deck_database['15ft']
        Allow_Load_16ft = Deck_database['16ft']
        Deck_Design = Deck_database['Design']
        Deck_Weight = Deck_database['Deck Weight (psf)']

        #Steel Member Data Pulled from '"AISC STEEL MANUAL V16 (Modified) Sorted.csv"
        shape_label = np.array(W_Shapes['AISC_Manual_Label'])
        SW = W_Shapes['W']
        A = np.array(W_Shapes['A'])
        d = np.array(W_Shapes['d']).astype(float)
        bf_over_2tf = W_Shapes['bf/2tf']
        Sx = W_Shapes['Sx'].astype(float)
        T = W_Shapes['T'].astype(float)
        Jc = W_Shapes['J'].astype(float)
        rts = W_Shapes['rts'].astype(float)
        ho = W_Shapes['ho'].astype(float)
        Z = W_Shapes['Zx [in3]']
        bf = W_Shapes['bf'].astype(float)
        tf = W_Shapes['tf'].astype(float)
        tw = W_Shapes['tw'].astype(float)
        PhiMpx = W_Shapes['PhiMpx']
        PhiBF = W_Shapes['PhiBF']
        Lp = W_Shapes['Lp [ft]']
        Lr = W_Shapes['Lr [ft]']
        Ix = W_Shapes['Ix [in4]']
        PhiVnx = W_Shapes['PhiVnx']
        Economic = W_Shapes['Economic']
        Groups = W_Shapes['Grouping #']
        Ingroup = W_Shapes['# in Grouping']

        #--------------------------------------------------------------------
        #------------------------Deck Sizer----------------------------------
        #--------------------------------------------------------------------

        # Define dictionaries
        span_data_dict = {
            1: Clear_Span_1,
            2: Clear_Span_2,
            3: Clear_Span_3
        }

        load_data_dict = {
            4: Allow_Load_4ft,
            5: Allow_Load_5ft,
            6: Allow_Load_6ft,
            7: Allow_Load_7ft,
            8: Allow_Load_8ft,
            9: Allow_Load_9ft,
            10: Allow_Load_10ft,
            11: Allow_Load_11ft,
            12: Allow_Load_12ft,
            13: Allow_Load_13ft,
            14: Allow_Load_14ft,
            15: Allow_Load_15ft,
            16: Allow_Load_16ft
        }

        Building_Use_dict = {
            1: ('Paper Office', 11, 0.5, 50, 0.03),
            2: ('Electronic Office', 7, 0.5, 50, 0.025),
            3: ('Residence', 6, 0.5, 40, 0.02),
            4: ('Assembly Area', 0, 0.5, 100, 0.02),
            5: ('Shopping Mall', 0, 1.5, 100, 0.02) #Cut Shopping Malls
        }
        Building_Use, live_load_vibrations, tolerance_limit, LL, DR = Building_Use_dict.get(Building_Use_num, ('', '', '', ''))

        
        if Conc_Type_num == 0:
            Conc_Type = 'NW'
            Conc_ECV = 0.133 #Concrete Embodied Carbon Value
            conc_density = 145 #unit-weight of concrete [pcf]
        else:
            Conc_Type = 'LW'
            Conc_ECV = 0.307 #Concrete Embodied Carbon Value
            conc_density = 110 #unit-weight of concrete [pcf]

        #This is additional Concrete load as a result of Additional Concrete Topping because the Vulcraft tables factor out concrete self weight of set thicknesses
        if Conc_Type == 'NW':
            Additional_Topping_Load = 145 * (Additional_Topping / 12) #[psf]
        if Conc_Type == 'LW':
            Additional_Topping_Load = 110 * (Additional_Topping / 12) #[psf]
        
        # Superimposed Dead Load [psf]
        SDL = 20 #Assumed for each building use
        Conc_Load = LL + SDL + Additional_Topping_Load

        # Concrete Type Check
        if len(Conc_Type) != len(Conc_Type_Table):
            Conc_Type = list(repeat(Conc_Type, 90))
        Conc_Type_Check = Conc_Type == Conc_Type_Table

        # Deck Type Check
        Deck_Type_Check = Deck_Type == Deck_Type_Table

        # Span Check
        Span_Data = span_data_dict.get(Clear_Spans)
        Span_Check = Span_Data >= infill_spacing

        # Span Check
        if infill_spacing_rounded < 4:
            infill_spacing_rounded = 4
        Load_Data = load_data_dict.get(infill_spacing_rounded)
        if Load_Data is None:
            print("No deck can be sized")
            deck_pass_or_fail = 0  # fails
            return
        elif Load_Data.empty:
            print("Load Data is empty")
            deck_pass_or_fail = 0  # fails
            return
        else:
            deck_pass_or_fail = 1  # passes

        Load_Check = Conc_Load <= Load_Data

        # Set Intersection of all deck checks
        Deck_Check = Conc_Type_Check & Deck_Type_Check & Span_Check & Load_Check

        # Filter Decks based on Deck_Check
        filtered_Decks = Deck_database[Deck_Check]
        
        # Check if filtered_Decks is empty
        if filtered_Decks.empty:
            print("No deck can be sized")
            deck_pass_or_fail = 0 #fails
            return
        else:
            deck_pass_or_fail = 1 #passes
        
        # Select Lightest Deck
        lowest_Slab_Weight = filtered_Decks['Concrete + Deck (psf)'].idxmin()
        deck_pass_or_fail = []
        selected_deck = filtered_Decks.loc[lowest_Slab_Weight, 'Design']

        # New Slab Weight
        selected_deck_slab_SW = filtered_Decks.loc[lowest_Slab_Weight, 'Concrete + Deck (psf)']
        Final_Slab_SW = selected_deck_slab_SW + Additional_Topping_Load

        # New Slab Thickness
        selected_deck_slab_thickness = filtered_Decks.loc[lowest_Slab_Weight, 'Total'] 
        selected_deck_topping_thickness = filtered_Decks.loc[lowest_Slab_Weight, 'Topping'] # Selected Topping Thickness without Addtional [in]
        selected_deck_thickness = filtered_Decks.loc[lowest_Slab_Weight, 'Deck Type'] # Selected Deck Thickness [in]
        selected_deck_SW = filtered_Decks.loc[lowest_Slab_Weight, 'Deck Weight (psf)']
        Final_Slab_Thickness = selected_deck_topping_thickness + Additional_Topping # Total Selected Slab Thickness [in]
        Final_Deck_Slab_Thickness = Final_Slab_Thickness + selected_deck_thickness # Total Selected Floor System Thicknes [in]
        
        #New Dead Load
        DL = Final_Slab_SW + SDL

        #Average Concrete Depth in Flute
        if Deck_Type == 1.5:
            Conc_Depth_in_Flute = 0.532414
        if Deck_Type == 2:
            Conc_Depth_in_Flute = 1
        if Deck_Type == 3:
            Conc_Depth_in_Flute = 1.5
        
        effective_conc_depth = selected_deck_topping_thickness + Conc_Depth_in_Flute + Additional_Topping

        #----------------------------------------------------------------------
        #----------------------Composite Values--------------------------------
        #----------------------------------------------------------------------

        # Basic parameters
        tc = selected_deck_topping_thickness
        hr = Deck_Type

        #------------------------Infill Values----------------------------------

        A_II = np.array(A)[:, np.newaxis]
        bf_II = np.array(bf)[:, np.newaxis]
        tf_II = np.array(tf)[:, np.newaxis]
        tw_II = np.array(tw)[:, np.newaxis]
        Z_II = np.array(Z)[:, np.newaxis]
        d_II = np.array(d)[:, np.newaxis]
        Ix_II = np.array(Ix)[:, np.newaxis]

        # Effective width calculation
        b_eff_II = min(Infill_Span * 12 * 2 / 8, infill_spacing * 12 / 2 * 2)

        # Force calculations
        Vc_prime_II = np.array(0.85 * f_prime_c * b_eff_II * tc)
        Vs_prime_II = np.array(A_II * Fy)
        V_prime_II = np.array(np.minimum(Vc_prime_II, Vs_prime_II))

        # Stud calculations
        stud_dia_II = 0.75
        studs_per_rib_II = np.expand_dims(np.array([1, 2, 3]), axis=(0, 1))
        Rg_II = np.expand_dims(np.array([1.0, 0.85, 0.7]), axis=(0, 1))
        Rp_II = 0.6
        Fu_II = 65
        A_stud_II = (ms.pi * stud_dia_II ** 2 / 4)
        Ec_II = conc_density ** 1.5 * ms.sqrt(f_prime_c)
        Qn_II = np.minimum(0.5 * A_stud_II * ms.sqrt(f_prime_c * Ec_II), Rg_II * Rp_II * A_stud_II * Fu_II)

        # Stud count calculations
        max_studs_half_II = np.floor(Infill_Span * studs_per_rib_II / 2)
        max_PCC_II = np.minimum(max_studs_half_II * Qn_II / V_prime_II[:, np.newaxis] * 100, 100)
        req_studs_half_II = percent_comp * V_prime_II[:, np.newaxis] / Qn_II / 100
        req_studs_half_filter_II = np.where(max_studs_half_II <= req_studs_half_II, np.nan, req_studs_half_II)
        req_studs_II = np.ceil(req_studs_half_filter_II * 2)
        min_req_studs_II = np.nanmin(req_studs_II, axis=2)

        # Minimum studs per rib calculation
        min_studs_per_rib_II = np.where(min_req_studs_II == req_studs_II[:, :, 0], 1, np.nan)
        min_studs_per_rib_II = np.where(min_req_studs_II == req_studs_II[:, :, 1], 2, min_studs_per_rib_II)
        min_studs_per_rib_II = np.where(min_req_studs_II == req_studs_II[:, :, 2], 3, min_studs_per_rib_II)

        stud_capacity_II = np.where(min_req_studs_II == req_studs_II[:, :, 0], req_studs_II[:, :, 0] * Qn_II[:, :, 0], np.nan)
        stud_capacity_II = np.where(min_req_studs_II == req_studs_II[:, :, 1], req_studs_II[:, :, 1] * Qn_II[:, :, 1], stud_capacity_II)
        stud_capacity_II = np.where(min_req_studs_II == req_studs_II[:, :, 2], req_studs_II[:, :, 2] * Qn_II[:, :, 2], stud_capacity_II)

        selected_Qn_II = np.where(min_req_studs_II == req_studs_II[:, :, 0], Qn_II[:, :, 0], np.nan)
        selected_Qn_II = np.where(min_req_studs_II == req_studs_II[:, :, 1], Qn_II[:, :, 1], selected_Qn_II)
        selected_Qn_II = np.where(min_req_studs_II == req_studs_II[:, :, 2], Qn_II[:, :, 2], selected_Qn_II)

        # 100% composite calculations
        if percent_comp == 100:
            Sigma_Qn_II = percent_comp / 100 * V_prime_II
            Case_1_II = (Vs_prime_II <= Vc_prime_II)
            a_1_II = np.minimum(tc, A_II * Fy / (0.85 * f_prime_c * b_eff_II))
            Cc_1_II = 0.85 * f_prime_c * b_eff_II * a_1_II
            Y2_1_II = Final_Deck_Slab_Thickness - a_1_II / 2
            Phi_Mn_1_II = np.where(Case_1_II, 0.9 * (A_II * Fy * (d_II / 2 + Y2_1_II)) / 12, np.nan)

            Cs_II = (A_II * Fy - Cc_1_II) / 2
            Asc_II = np.array(Cs_II / Fy)
            A_flange_II = np.array(bf_II * tf_II)

            Case_2a_II = (Vs_prime_II > Vc_prime_II) & (Asc_II < A_flange_II)
            Cc_2a_II = 0.85 * f_prime_c * b_eff_II * tc
            T_2a_II = A_II * Fy
            Y1_2a_II = (A_II * Fy - Cc_2a_II) / (2 * bf_II * Fy)
            Y2_2a_II = hr + tc / 2
            Phi_Mn_2a_II = np.where(Case_2a_II, 0.9 * (Cc_2a_II * (Y2_2a_II + Y1_2a_II / 2) + T_2a_II * (d_II / 2 - Y1_2a_II / 2)) / 12, np.nan)

            Case_2b_II = (Vs_prime_II > Vc_prime_II) & (Asc_II >= A_flange_II)
            Cc_2b_II = 0.85 * f_prime_c * b_eff_II * tc
            Y1_2b_II = d_II / 2 - Cc_2b_II / (2 * tw_II * Fy)
            Y2_2b_II = hr + tc / 2
            Ts1_2b_II = 2 * tw_II * Fy * (d_II / 2 - Y1_2b_II)
            Mp_2b_II = Fy * Z_II
            Phi_Mn_2b_II = np.where(Case_2b_II, 0.9 * (Cc_2b_II * (Y2_2b_II + d_II / 2) - Ts1_2b_II * ((d_II / 2 - Y1_2b_II) / 2) + Mp_2b_II) / 12, np.nan)

            Phi_Mn_II = np.nanmax(np.stack([Phi_Mn_1_II, Phi_Mn_2a_II, Phi_Mn_2b_II]), axis=0)

            # Initialize Y1 and Y2
            Y1_II = np.zeros_like(Case_1_II, dtype=float)
            Y2_II = np.full_like(Case_1_II, np.nan, dtype=float)

            Y1_II = np.where(Case_1_II, 0, Y1_II)
            Y2_II = np.where(Case_1_II, Y2_1_II, Y2_II)

            Y1_II = np.where(Case_2a_II, Y1_2a_II, Y1_II)
            Y2_II = np.where(Case_2a_II, Y2_2a_II, Y2_II)

            Y1_II = np.where(Case_2b_II, Y1_2b_II, Y1_II)
            Y2_II = np.where(Case_2b_II, Y2_2b_II, Y2_II)

            case_label_II = np.where(Case_1_II, 'Case 1', 
                np.where(Case_2a_II, 'Case 2a', 
                np.where(Case_2b_II, 'Case 2b', 'Unknown Case')))
            
        # partial composite calculation
        elif 25 <= percent_comp < 100:
            Sigma_Qn_II = percent_comp / 100 * V_prime_II
            a_3_II = np.minimum(Sigma_Qn_II / (0.85 * f_prime_c * b_eff_II), tc)
            Cc_3_II = np.minimum(Sigma_Qn_II, 0.85 * f_prime_c * b_eff_II * a_3_II)
            Cs_3a_II = (A_II * Fy - Cc_3_II) / 2
            T_3a_II = A_II * Fy
            Y1_3a_II = (A_II * Fy - Cc_3_II) / (2 * bf_II * Fy)
            Y2_3a_II = hr + tc - a_3_II / 2
            Case_3a_II = Y1_3a_II <= tf_II
            Phi_Mn_3a_II = np.where(Case_3a_II, 0.9 * (Cc_3_II * (Y2_3a_II + Y1_3a_II / 2) + T_3a_II * (d_II / 2 - Y1_3a_II / 2)) / 12, np.nan)

            Y1_3b_II = d_II / 2 - Cc_3_II / (2 * tw_II * Fy)
            Y2_3b_II = hr + tc / 2
            Ts1_3b_II = 2 * tw_II * Fy * (d_II / 2 - Y1_3b_II)
            Case_3b_II = Y1_3a_II > tf_II
            Mp_3b_II = Fy * Z_II
            Phi_Mn_3b_II = np.where(Case_3b_II, 0.9 * (Cc_3_II * (Y2_3b_II + d_II / 2) - Ts1_3b_II * ((d_II / 2 - Y1_3b_II) / 2) + Mp_3b_II) / 12, np.nan)

            Phi_Mn_II = np.nanmax(np.stack([Phi_Mn_3a_II, Phi_Mn_3b_II]), axis=0)

            # Initialize Y1 and Y2
            Y1_II = np.zeros_like(Case_3a_II, dtype=float)
            Y2_II = np.full_like(Case_3a_II, np.nan, dtype=float)

            Y1_II = np.where(Case_3a_II, Y1_3a_II, Y1_II)
            Y2_II = np.where(Case_3a_II, Y2_3a_II, Y2_II)

            Y1_II = np.where(Case_3b_II, Y1_3b_II, Y1_II)
            Y2_II = np.where(Case_3b_II, Y2_3b_II, Y2_II)

            case_label_II = np.where(Case_3a_II, 'Case 3a', 
                np.where(Case_3b_II, 'Case 3b', 'Unknown Case'))

        else:
            num_studs_II = np.nan

        # PCC check calculations
        PCC_check_II = np.max(max_PCC_II >= percent_comp, axis=2)
        Phi_Mn_filtered_II = np.where(PCC_check_II, Phi_Mn_II, np.nan)

        # Lower Bound Moment of Inertia calculations
        a_II = np.minimum(Sigma_Qn_II / (0.85 * f_prime_c * b_eff_II), tc)
        d1_II = tc + hr - a_II / 2
        d3_II = d_II/2
        Y_ENA_II = (A_II * d3_II + (Sigma_Qn_II / Fy) * (2 * d3_II + d1_II)) / (A_II + (Sigma_Qn_II / Fy))
        I_LB_II = Ix_II + A_II * (Y_ENA_II - d3_II) ** 2 + (Sigma_Qn_II / Fy) * (2 * d3_II + d1_II - Y_ENA_II) ** 2

        #------------------------Girder Values----------------------------------

        A_IG = np.array(A)[np.newaxis, :, np.newaxis]
        bf_IG = np.array(bf)[np.newaxis, :, np.newaxis]
        tf_IG = np.array(tf)[np.newaxis, :, np.newaxis]
        tw_IG = np.array(tw)[np.newaxis, :, np.newaxis]
        Z_IG = np.array(Z)[np.newaxis, :, np.newaxis]
        d_IG = np.array(d)[np.newaxis, :, np.newaxis]
        Ix_IG = np.array(Ix)[np.newaxis, :, np.newaxis]

        # Effective width calculation
        b_eff_IG = min(Girder_Span * 12 * 2 / 8, Infill_Span * 12 / 2 * 2)

        # Force calculations
        Vc_prime_IG = np.array(0.85 * f_prime_c * b_eff_IG * tc)
        Vs_prime_IG = np.array(A_IG * Fy)
        V_prime_IG = np.array(np.minimum(Vc_prime_IG, Vs_prime_IG))

        # Stud calculations
        stud_dia_IG = 0.75
        studs_per_rib_IG = np.expand_dims(np.array([1, 2, 3]), axis=(0, 1))
        Rg_IG = np.expand_dims(np.array([1.0, 0.85, 0.7]), axis=(0, 1))
        Rp_IG = 0.6
        Fu_IG = 65
        A_stud_IG = (ms.pi * stud_dia_IG ** 2 / 4)
        Ec_IG = conc_density ** 1.5 * ms.sqrt(f_prime_c)
        Qn_IG = np.minimum(0.5 * A_stud_IG * ms.sqrt(f_prime_c * Ec_IG), Rg_IG * Rp_IG * A_stud_IG * Fu_IG)

        # Stud count calculations
        max_studs_half_IG = np.floor(Girder_Span * studs_per_rib_IG / 2)
        max_PCC_IG = np.minimum(max_studs_half_IG * Qn_IG / V_prime_IG * 100, 100)
        req_studs_half_IG = percent_comp * V_prime_IG / Qn_IG / 100
        req_studs_half_filter_IG = np.where(max_studs_half_IG <= req_studs_half_IG, np.nan, req_studs_half_IG)
        req_studs_IG = np.ceil(req_studs_half_filter_IG * 2)
        min_req_studs_IG = np.nanmin(req_studs_IG, axis=2)

        # Minimum studs per rib calculation
        min_studs_per_rib_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 0], 1, np.nan)
        min_studs_per_rib_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 1], 2, min_studs_per_rib_IG)
        min_studs_per_rib_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 2], 3, min_studs_per_rib_IG)

        stud_capacity_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 0], req_studs_IG[:, :, 0] * Qn_IG[:, :, 0], np.nan)
        stud_capacity_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 1], req_studs_IG[:, :, 1] * Qn_IG[:, :, 1], stud_capacity_IG)
        stud_capacity_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 2], req_studs_IG[:, :, 2] * Qn_IG[:, :, 2], stud_capacity_IG)

        selected_Qn_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 0], Qn_IG[:, :, 0], np.nan)
        selected_Qn_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 1], Qn_IG[:, :, 1], selected_Qn_IG)
        selected_Qn_IG = np.where(min_req_studs_IG == req_studs_IG[:, :, 2], Qn_IG[:, :, 2], selected_Qn_IG)

        # 100% composite calculations
        if percent_comp == 100:
            Sigma_Qn_IG = percent_comp / 100 * V_prime_IG
            Case_1_IG = (Vs_prime_IG <= Vc_prime_IG)
            a_1_IG = np.minimum(tc, A_IG * Fy / (0.85 * f_prime_c * b_eff_IG))
            Cc_1_IG = 0.85 * f_prime_c * b_eff_IG * a_1_IG
            Y2_1_IG = Final_Deck_Slab_Thickness - a_1_IG / 2
            Phi_Mn_1_IG = np.where(Case_1_IG, 0.9 * (A_IG * Fy * (d_IG / 2 + Y2_1_IG)) / 12, np.nan)

            Cs_IG = (A_IG * Fy - Cc_1_IG) / 2
            Asc_IG = np.array(Cs_IG / Fy)
            A_flange_IG = np.array(bf_IG * tf_IG)

            Case_2a_IG = (Vs_prime_IG > Vc_prime_IG) & (Asc_IG < A_flange_IG)
            Cc_2a_IG = 0.85 * f_prime_c * b_eff_IG * tc
            T_2a_IG = A_IG * Fy
            Y1_2a_IG = (A_IG * Fy - Cc_2a_IG) / (2 * bf_IG * Fy)
            Y2_2a_IG = hr + tc / 2
            Phi_Mn_2a_IG = np.where(Case_2a_IG, 0.9 * (Cc_2a_IG * (Y2_2a_IG + Y1_2a_IG / 2) + T_2a_IG * (d_IG / 2 - Y1_2a_IG / 2)) / 12, np.nan)

            Case_2b_IG = (Vs_prime_IG > Vc_prime_IG) & (Asc_IG >= A_flange_IG)
            Cc_2b_IG = 0.85 * f_prime_c * b_eff_IG * tc
            Y1_2b_IG = d_IG / 2 - Cc_2b_IG / (2 * tw_IG * Fy)
            Y2_2b_IG = hr + tc / 2
            Ts1_2b_IG = 2 * tw_IG * Fy * (d_IG / 2 - Y1_2b_IG)
            Mp_2b_IG = Fy * Z_IG
            Phi_Mn_2b_IG = np.where(Case_2b_IG, 0.9 * (Cc_2b_IG * (Y2_2b_IG + d_IG / 2) - Ts1_2b_IG * ((d_IG / 2 - Y1_2b_IG) / 2) + Mp_2b_IG) / 12, np.nan)

            Phi_Mn_IG = np.nanmax(np.stack([Phi_Mn_1_IG, Phi_Mn_2a_IG, Phi_Mn_2b_IG]), axis=0)

            # Initialize Y1 and Y2 with default values
            Y1_IG = np.zeros_like(Case_1_IG, dtype=float)
            Y2_IG = np.full_like(Case_1_IG, np.nan, dtype=float)

            Y1_IG = np.where(Case_1_IG, 0, Y1_IG)
            Y2_IG = np.where(Case_1_IG, Y2_1_IG, Y2_IG)

            Y1_IG = np.where(Case_2a_IG, Y1_2a_IG, Y1_IG)
            Y2_IG = np.where(Case_2a_IG, Y2_2a_IG, Y2_IG)

            Y1_IG = np.where(Case_2b_IG, Y1_2b_IG, Y1_IG)
            Y2_IG = np.where(Case_2b_IG, Y2_2b_IG, Y2_IG)

            case_label_IG = np.where(Case_1_IG, 'Case 1', 
                np.where(Case_2a_IG, 'Case 2a', 
                np.where(Case_2b_IG, 'Case 2b', 'Unknown Case')))

        # partial composite calculations
        elif 25 <= percent_comp < 100:
            Sigma_Qn_IG = percent_comp / 100 * V_prime_IG
            a_3_IG = np.minimum(Sigma_Qn_IG / (0.85 * f_prime_c * b_eff_IG), tc)
            Cc_3_IG = np.minimum(Sigma_Qn_IG, 0.85 * f_prime_c * b_eff_IG * a_3_IG)
            Cs_3a_IG = (A_IG * Fy - Cc_3_IG) / 2
            T_3a_IG = A_IG * Fy
            Y1_3a_IG = (A_IG * Fy - Cc_3_IG) / (2 * bf_IG * Fy)
            Y2_3a_IG = hr + tc - a_3_IG / 2
            Case_3a_IG = Y1_3a_IG <= tf_IG
            Phi_Mn_3a_IG = np.where(Case_3a_IG, 0.9 * (Cc_3_IG * (Y2_3a_IG + Y1_3a_IG / 2) + T_3a_IG * (d_IG / 2 - Y1_3a_IG / 2)) / 12, np.nan)

            Y1_3b_IG = d_IG / 2 - Cc_3_IG / (2 * tw_IG * Fy)
            Y2_3b_IG = hr + tc / 2
            Ts1_3b_IG = 2 * tw_IG * Fy * (d_IG / 2 - Y1_3b_IG)
            Case_3b_IG = Y1_3a_IG > tf_IG
            Mp_3b_IG = Fy * Z_IG
            Phi_Mn_3b_IG = np.where(Case_3b_IG, 0.9 * (Cc_3_IG * (Y2_3b_IG + d_IG / 2) - Ts1_3b_IG * ((d_IG / 2 - Y1_3b_IG) / 2) + Mp_3b_IG) / 12, np.nan)

            Phi_Mn_IG = np.nanmax(np.stack([Phi_Mn_3a_IG, Phi_Mn_3b_IG]), axis=0)

            # Initialize Y1 and Y2
            Y1_IG = np.zeros_like(Case_3a_IG, dtype=float)
            Y2_IG = np.full_like(Case_3a_IG, np.nan, dtype=float)

            Y1_IG = np.where(Case_3a_IG, Y1_3a_IG, Y1_IG)
            Y2_IG = np.where(Case_3a_IG, Y2_3a_IG, Y2_IG)

            Y1_IG = np.where(Case_3b_IG, Y1_3b_IG, Y1_IG)
            Y2_IG = np.where(Case_3b_IG, Y2_3b_IG, Y2_IG)

            case_label_IG = np.where(Case_3a_IG, 'Case 3a', 
                np.where(Case_3b_IG, 'Case 3b', 'Unknown Case'))

        else:
            print('PCC below 25% can not be composite')

        # PCC check calculations
        PCC_check_IG = max_PCC_IG >= percent_comp 
        PCC_check_flat_IG = np.max(max_PCC_IG, axis=2) >= percent_comp
        Phi_Mn_filtered_IG = np.where(PCC_check_flat_IG, Phi_Mn_IG, np.nan)

        # Lower Bound Moment of Inertia calculations
        a_IG = np.minimum(Sigma_Qn_IG / (0.85 * f_prime_c * b_eff_IG), tc)
        d1_IG = tc + hr - a_IG / 2
        d3_IG = d_IG/2
        Y_ENA_IG = (A_IG * d3_IG + (Sigma_Qn_IG / Fy) * (2 * d3_IG + d1_IG)) / (A_IG + (Sigma_Qn_IG / Fy))
        I_LB_IG = Ix_IG + A_IG * (Y_ENA_IG - d3_IG) ** 2 + (Sigma_Qn_IG / Fy) * (2 * d3_IG + d1_IG - Y_ENA_IG) ** 2

        # Need to flatten some IG variables used in later calcs
        Phi_Mn_IG = Phi_Mn_IG.flatten()
        I_LB_IG = I_LB_IG.flatten()
        Y1_IG = Y1_IG.flatten()
        Y2_IG = Y2_IG.flatten()
        case_label_IG = case_label_IG.flatten()
        PCC_check_flat_IG = PCC_check_flat_IG.flatten()
        min_req_studs_IG = min_req_studs_IG.flatten()
        min_studs_per_rib_IG = min_studs_per_rib_IG.flatten()

        #--------------------------------------------------------------------
        #----------------------Interior Infill-------------------------------
        #--------------------------------------------------------------------

        #----------------------Initial Values--------------------------------

        SW_II = np.array(SW)[:, np.newaxis]  # Shape: (289, 1)
        infill_Ix = np.array(Ix)[:, np.newaxis]
        I_LB_II = np.array(I_LB_II)

        # Trib width of Interior Infill
        Trib_II = np.array(Girder_Span / (NumInfill + 1)).reshape(1,) # Shape: ()

        # Interior Infill Dead Load w/ Self Weight (psf)
        DL_II = DL + (SW_II / Trib_II)  # Shape: (289, 1)

        # Interior Infill Dead Load (klf) = W_DL_II
        W_DL_II = (DL_II * Trib_II) / 1000.0  # Shape: (289, 1)

        # Interior Infill Live Load (klf) = W_LL_II
        W_LL_II = np.array((LL * Trib_II) / 1000.0).reshape(1,) # Shape: ()

        # Interior Infill Total Load (klf) = W_TL_II
        W_TL_II = ((DL_II + LL) * Trib_II) / 1000.0  # Shape: (289, 1)

        # Interior Infill Construction Dead Load (klf) = W_Con_DL_II
        W_Con_DL_II = (Final_Slab_SW * Trib_II + SW_II) / 1000.0  # Shape: (289, 1)

        # Interior Infill Construction Live Load (klf) = W_Con_LL_II
        W_Con_LL_II = np.array((20 * Trib_II) / 1000.0).reshape(1,) # Shape: ()

        # Interior Infill Composite Live Load (klf) = W_CLL_II
        W_CLL_II = np.array(W_LL_II * 0.5).reshape(1,1) # Shape: ()

        # Interior Infill Precomposite Factored Load (klf) = W_PC_II
        W_PC_II_LC1 = 1.4 * W_Con_DL_II
        W_PC_II_LC2 = 1.2 * W_Con_DL_II + 1.6 * W_Con_LL_II
        W_PC_II = np.maximum(W_PC_II_LC1, W_PC_II_LC2)  # Shape: (289, 1)

        # Interior Infill Shear Dead Load (kips) = V_DL_II
        V_DL_II = W_DL_II * Infill_Span / 2.0  # Shape: (289, 1)

        # Interior Infill Shear Live Load (kips) = V_LL_II
        V_LL_II = np.array(W_LL_II * Infill_Span / 2.0).reshape(1,) # Shape: ()

        # Interior Infill Shear Self Weight
        V_SW_II = (SW_II / Trib_II) * Infill_Span / 2.0 * Trib_II  # Shape: (289, 1)

        # Interior Infill Shear Factored Load (kips) = V_FL_II
        V_II_LC1 = 1.4 * V_DL_II
        V_II_LC2 = 1.2 * V_DL_II + 1.6 * V_LL_II
        V_FL_II = np.maximum(V_II_LC1, V_II_LC2)  # Shape: (289, 1)

        # Interior Infill Construction Dead Load Shear (kips) = V_Con_DL_II
        V_Con_DL_II = W_Con_DL_II * Infill_Span / 2.0  # Shape: (289, 1)

        # Interior Infill Pre-Composite Factored Shear (kips) = V_PC_II
        V_PC_II = W_PC_II * Infill_Span / 2.0  # Shape: (289, 1)

        # Interior Infill Composite Live Load Shear (kips) = V_CLL_II
        V_CLL_II = np.array(W_CLL_II * Infill_Span / 2.0).reshape(1,) # Shape: ()

        # Interior Infill Moment Dead Load (kips * ft ) = M_DL_II
        M_DL_II = W_DL_II * Infill_Span ** 2 / 8  # Shape: (289, 1)

        # Interior Infill Moment Live Load (kips * ft ) = M_LL_II
        M_LL_II = np.array(W_LL_II * Infill_Span ** 2 / 8).reshape(1,) # Shape: ()

        # Interior Infill Moment Factored Load (kips * ft ) = M_FL_II
        M_II_LC1 = 1.4 * M_DL_II
        M_II_LC2 = 1.2 * M_DL_II + 1.6 * M_LL_II
        M_FL_II = np.maximum(M_II_LC1, M_II_LC2)  # Shape: (289, 1)

        # Interior Infill Pre-Composite Moment Factored Load (kips * ft ) = M_PC_II
        M_PC_II = W_PC_II * Infill_Span ** 2 / 8  # Shape: (289, 1)

        # Interior Infill Total Load Deflection DEF_TL_II
        DEF_TL_II = (5*W_TL_II*(Infill_Span**4)*1728)/(384*Es*infill_Ix)  # Shape: (289, 1)

        # Interior Infill Live Load Deflection DEF_LL_II
        DEF_LL_II = (5*W_LL_II*(Infill_Span**4)*1728)/(384*Es*infill_Ix) # Shape: ()

        # Interior Infill Construction Deflection DEF_Con_II (no LL, no combos)
        DEF_Con_II = (5*W_Con_DL_II*(Infill_Span**4)*1728)/(384*Es*infill_Ix)  # Shape: (289, 1)

        # Interior Infill Composite Live Load Deflection DEF_CLL_II
        DEF_CLL_II = (5*W_CLL_II*(Infill_Span**4)*1728)/(384*Es*I_LB_II)# Shape: (289, 1) 

        # Interior Infill Check to make sure Camber is greater than 0.75" and the Span is greater than 24'
        camber_check = (np.around(0.8 * DEF_Con_II / 0.25) * 0.25 > 0.625) & (Infill_Span >= 24)  # Shape: (289, 1)

        # Interior Infill Camber based on 0.8 * Construction Deflection, limited by l/360
        camber_II = np.minimum(np.where(camber_check, np.around(0.8 * DEF_Con_II / 0.25) * 0.25, 0), 4) # Shape: (289, 1) Not cambering more than l/360

        # Interior Infill Composite Dead Load Deflection
        DEF_CDL_II = DEF_Con_II - camber_II

        # Interior Infill Composite Total Load Deflection
        DEF_CTL_II = DEF_CDL_II + DEF_CLL_II

        #----------------------Filtering Sizes--------------------------------

        # Pre-composite checks
        PhiVnx_II = np.expand_dims(PhiVnx, axis = 1)
        PhiMpx_II = np.expand_dims(PhiMpx, axis = 1)
        pre_composite_V_check_II = V_PC_II <= PhiVnx_II
        pre_composite_M_check_II = M_PC_II <= PhiMpx_II

        # Composite checks
        composite_V_check_II = V_FL_II <= PhiVnx_II
        composite_M_check_II = M_FL_II <= Phi_Mn_II

        # Deflecition checks
        DL_DEF_check_II = np.array(DEF_CDL_II <= min(Infill_Span * 12 / 360, 1))
        LL_DEF_check_II = np.array(DEF_CLL_II <= min(Infill_Span * 12 / 360, 1))
        TL_DEF_check_II = np.array(DEF_CTL_II <= Infill_Span * 12 / 240)

        # Economic Size Condition
        Eco = np.expand_dims(np.array(Economic == 1), axis = 0)

        # Combined Check
        combined_check_II = (pre_composite_V_check_II &
                             pre_composite_M_check_II & 
                             composite_V_check_II & 
                             composite_M_check_II & 
                             DL_DEF_check_II & 
                             LL_DEF_check_II & 
                             TL_DEF_check_II&
                             PCC_check_II
                            )

        # Sorting
        Groups_array = np.array(Groups)[:, np.newaxis]
        Ingroup_array = np.array(Ingroup)[:, np.newaxis]

        infill_sizes = np.expand_dims(shape_label, axis = 1).astype(str)
        infill_depth = np.expand_dims(d, axis = 1)
        infill_area = np.expand_dims(A, axis = 1)
        infill_sw = np.expand_dims(SW, axis = 1)

        #--------------------------------------------------------------------
        #----------------------Interior Girder-------------------------------
        #--------------------------------------------------------------------

        #----------------------Initial Values--------------------------------

        I_LB_IG = np.array(I_LB_IG).flatten()

        #Self Weight of Interior Inifll    
        SW_IG = np.expand_dims(np.array(SW), axis = (0)) # Shape: (1, 289)

        #Trib width of Interior Girder
        Trib_IG = Infill_Span # Shape: (1, )
        Trib_II = Girder_Span / (NumInfill + 1) # Shape: (1, )

        #Total SW of Infills on Interior Girders (kips * ft)
        Total_SW_II = ((SW_II / 1000) * NumInfill * Trib_IG / Girder_Span) # Shape: (289, 1)

        # Interior Girder Construction Dead Load (klf) = W_Con_DL_IG
        W_Con_DL_IG = (Final_Slab_SW * Trib_IG + SW_IG) / 1000.0 + Total_SW_II # Shape: (289, 289)

        # Interior Girder Construction Live Load (klf) = W_Con_LL_IG
        W_Con_LL_IG = (20 * Trib_IG) / 1000.0 # Shape: (1, )

        # Interior Girder Precomposite Factored Load (klf) = W_PC_IG
        W_PC_IG_LC1 = 1.4 * W_Con_DL_IG
        W_PC_IG_LC2 = 1.2 * W_Con_DL_IG + 1.6 * W_Con_LL_IG
        W_PC_IG = np.maximum(W_PC_IG_LC1, W_PC_IG_LC2) # Shape: (289, 289)

        #Interior Girder Shear Dead Load (kips) = V_DL_IG
        V_DL_IG = ((SW_IG * Girder_Span / 1000) + 2 * V_DL_II * NumInfill) / 2 # Shape: (289, 289)

        #Interior Girder Shear Dead Load (kips) = V_DL_IG
        V_LL_IG = (2 * V_LL_II * NumInfill) / 2 # Shape: (1, )
        
        #Interior Girder Pre-Composite Factored Load (kips) = V_PC_IG
        V_PC_IG = (2 * V_PC_II * NumInfill + SW_IG * Girder_Span / 1000) / 2 # Shape: (289, 289)

        # Interior Girder Composite Live Load Shear (kips) = V_CLL_IG
        V_CLL_IG = (2 * V_CLL_II * NumInfill) / 2 # Shape: (1, )

        #Interior Girder Shear Factored Load (kips * ft ) = V_FL_IG
        V_IG_LC1 = 1.4 * V_DL_IG
        V_IG_LC2 = 1.2 * V_DL_IG + 1.6 * V_LL_IG
        V_FL_IG = np.maximum(V_IG_LC1, V_IG_LC2) # Shape: (289, 289)

        #Interior Girder Moment from SW (kips*ft) = M_SW_IG
        M_SW_IG = np.array((SW_IG / 1000) * Girder_Span ** 2 / 8) # Shape: (1, 289)

        #Interior Girder Moment Dead Load (kips*ft) = M_DL_IG
        if NumInfill == 0:
            M_DL_IG = M_SW_IG
        if NumInfill == 1:
            M_DL_IG = M_SW_IG + (2 * V_DL_II * Girder_Span / 4) # Shape: (289, 289)
        if NumInfill == 2:
            M_DL_IG = M_SW_IG + (2 * V_DL_II * Trib_II) # Shape: (289, 289)
        if NumInfill == 3:
            M_DL_IG = M_SW_IG + (2 * (2 * V_DL_II * Girder_Span / 4)) # Shape: (289, 289)
        if NumInfill > 3:
            M_DL_IG = M_SW_IG + ((DL / 1000) * Trib_IG) * Girder_Span ** 2 / 8 + (Total_SW_II * Girder_Span ** 2 / 8) # Shape: (289, 289)

        #Interior Girder Moment Live Load (kips*ft) = M_DL_IG
        if NumInfill == 0: 
            M_LL_IG = 0 
        if NumInfill == 1:
            M_LL_IG = (2 * V_LL_II * Girder_Span / 4) # Shape: (1, )
        if NumInfill == 2:
            M_LL_IG = (2 * V_LL_II * Trib_II) # Shape: (1, )
        if NumInfill == 3:
            M_LL_IG = (2 * (2 * V_LL_II * Girder_Span / 4)) # Shape: (1, )
        if NumInfill > 3:
            M_LL_IG = ((LL / 1000) * Trib_IG) * Girder_Span ** 2 / 8 # Shape: (1, )

        #Interior Girder Moment Factored Load (kips * ft ) = M_FL_IG
        M_IG_LC1 = 1.4 * M_DL_IG
        M_IG_LC2 = 1.2 * M_DL_IG + 1.6 * M_LL_IG
        M_FL_IG = np.maximum(M_IG_LC1, M_IG_LC2) # Shape: (289, 289)

        # Interior Girder Pre-Composite Moment Factored Load (kips * ft ) = M_PC_IG
        if NumInfill == 0:
            M_PC_IG = M_SW_IG
        if NumInfill == 1:
            M_PC_IG = M_SW_IG + (2 * V_PC_II * Girder_Span / 4) # Shape: (289, 289)
        if NumInfill == 2:
            M_PC_IG = M_SW_IG + (2 * V_PC_II * Trib_II) # Shape: (289, 289)
        if NumInfill == 3:
            M_PC_IG = M_SW_IG + (2 * (2 * V_PC_II * Girder_Span / 4)) # Shape: (289, 289)
        if NumInfill > 3:
            M_PC_IG = W_PC_IG * Girder_Span ** 2 / 8 # Shape: (289, 289)

        # Expand Ix to allow for NumPy vectorization
        Ix_array = np.expand_dims(Ix, axis = 0)

        #Interior Girder Deflection Self Weight (in) = DEF_SW_IG
        DEF_SW_IG = (5 * (SW_IG / 1000) * (Girder_Span ** 4) * 1728) / (384 * Es * Ix_array) # Shape: (1, 289)
        
        #Interior Girder Live Load Deflection (in) = DEF_LL_IG
        if NumInfill == 0:
            DEF_LL_IG = 0
        if NumInfill == 1:
            DEF_LL_IG = ((2 * V_LL_II * (Girder_Span ** 3) * 1728) / (48 * Es * Ix))
        if NumInfill == 2:
            DEF_LL_IG = ((23 * 2 * V_LL_II * (Girder_Span ** 3) * 1728) / (648 * Es * Ix))
        if NumInfill == 3:
            DEF_LL_IG = ((22 * 2 * V_LL_II * (Girder_Span ** 3) * 1728) / (768 * 29000 * Ix)) + ((2 * V_LL_II * (Girder_Span ** 3) * 1728) / (48 * 29000 * Ix))
        if NumInfill > 3:
            DEF_LL_IG = (5 * ((LL / 1000) * Trib_IG) * (Girder_Span ** 4) * 1728) / (384 * Es * Ix)
        
        DEF_LL_IG = np.expand_dims(DEF_LL_IG, axis = 0) # Shape: (1, 289)

        #Interior Girder Total Load Deflection (in) = DEF_TL_IG
        if NumInfill == 0:
            DEF_TL_IG = DEF_SW_IG
        if NumInfill == 1:
            DEF_TL_IG = DEF_SW_IG + DEF_LL_IG + ((2 * V_DL_II * (Girder_Span ** 3) * 1728) / (48 * Es * Ix_array)) # Shape: (289, 289)
        if NumInfill == 2:
            DEF_TL_IG = DEF_SW_IG + DEF_LL_IG + ((23 * 2 * V_DL_II * (Girder_Span ** 3) * 1728) / (648 * Es * Ix_array)) # Shape: (289, 289)
        if NumInfill == 3:
            DEF_TL_IG = DEF_SW_IG + DEF_LL_IG + ((22 * 2 * V_DL_II * (Girder_Span ** 3) * 1728) / (768 * 29000 * Ix_array)) + ((2 * V_DL_II * (Girder_Span ** 3) * 1728) / (48 * 29000 * Ix_array)) # Shape: (289, 289)
        if NumInfill > 3:
            DEF_TL_IG = DEF_SW_IG + DEF_LL_IG + (5 * ((DL / 1000) * Trib_IG) * (Girder_Span ** 4) * 1728) / (384 * Es * Ix_array) # Shape: (289, 289)

        # Interior Girder Construction Deflection (in) = DEF_Con_IG (no LL, no combos)
        if NumInfill == 0:
            DEF_Con_IG = DEF_SW_IG
        if NumInfill == 1:
            DEF_Con_IG = DEF_SW_IG + ((2 * V_Con_DL_II * (Girder_Span ** 3) * 1728) / (48 * Es * Ix_array)) # Shape: (289, 289)
        if NumInfill == 2:
            DEF_Con_IG = DEF_SW_IG + ((23 * 2 * V_Con_DL_II * (Girder_Span ** 3) * 1728) / (648 * Es * Ix_array)) # Shape: (289, 289)
        if NumInfill == 3:
            DEF_Con_IG = DEF_SW_IG + ((22 * 2 * V_Con_DL_II * (Girder_Span ** 3) * 1728) / (768 * 29000 * Ix_array)) + ((2 * V_Con_DL_II * (Girder_Span ** 3) * 1728) / (48 * 29000 * Ix_array)) # Shape: (289, 289)
        if NumInfill > 3:
            DEF_Con_IG = (5 * W_Con_DL_IG * (Girder_Span ** 4) * 1728) / (384 * Es * Ix_array) # Shape: (289, 289)     

        # Interior Girder Composite Live Load Deflection (in) = DEF_CLL_IG
        if NumInfill == 0:
            DEF_CLL_IG = 0
        if NumInfill == 1:
            DEF_CLL_IG = ((2 * V_CLL_II * (Girder_Span ** 3) * 1728) / (48 * Es * I_LB_IG))
        if NumInfill == 2:
            DEF_CLL_IG = ((23 * 2 * V_CLL_II * (Girder_Span ** 3) * 1728) / (648 * Es * I_LB_IG))
        if NumInfill == 3:
            DEF_CLL_IG = ((22 * 2 * V_CLL_II * (Girder_Span ** 3) * 1728) / (768 * 29000 * I_LB_IG)) + ((2 * V_CLL_II * (Girder_Span ** 3) * 1728) / (48 * 29000 * I_LB_IG))
        if NumInfill > 3:
            DEF_CLL_IG = (5 * ((0.5 * LL / 1000) * Trib_IG) * (Girder_Span ** 4) * 1728) / (384 * Es * I_LB_IG)
        
        DEF_CLL_IG = np.expand_dims(DEF_CLL_IG, axis = 0) # Shape: (1, 289)

        # Interior Girder Check to make sure Camber is greater than 0.75" and the Span is greater than 24'
        camber_check_IG = (np.around(0.8 * DEF_Con_IG / 0.25) * 0.25 > 0.625) & (Girder_Span >= 24)

        # Interior Girder Camber based on 0.8 * Construction Deflection, limited by l/360
        camber_IG = np.minimum(np.where(camber_check_IG, np.round(0.8 * DEF_Con_IG / 0.25) * 0.25, 0), 4) # Shape: (289, 289)

        # Interior Infill Composite Dead Load Deflection (in) = DEF_CDL_IG
        DEF_CDL_IG = DEF_Con_IG - camber_IG # Shape: (289, 289)

        # Interior Girder Composite Total Load Deflection
        DEF_CTL_IG = DEF_CDL_IG + DEF_CLL_IG

        #Cb Values (Table 3-1)
        if NumInfill == 0:
            Cb = 1.14
        if NumInfill == 1:
            Cb = 1.67
        if NumInfill == 2:
            Cb = 1
        if NumInfill == 3:
            Cb = 1.11
        if NumInfill > 3:
            Cb = 1
    
        #----------------------Filtering Sizes--------------------------------

        #Flexure Check (Lb<=Lp)
        Lb = Girder_Span / (NumInfill + 1)
        Compact = Lb <= Lp
        Compact_PhiMn = np.where(Compact, PhiMpx, np.nan)

        #Flexure Check (Lp < Lb <=Lr)
        Noncompact1 = Lp < Lb
        Noncompact2 = Lb <= Lr
        Noncompact_LTB = Cb*(PhiMpx-PhiBF*(Lb-Lp))
        Noncompact_PhiMn = np.where(Noncompact1 & Noncompact2, np.minimum(Noncompact_LTB, PhiMpx), np.nan)

        #Flexure Check (Lr < Lb)
        Slender = Lb > Lr
        Fcr = ((Cb*(np.pi)**2*Es)/(Lb*12/rts)**2)*np.sqrt(1+0.078*(Jc/(Sx*ho))*(Lb*12/rts)**2)
        Slender_LTB = 0.9*((Fcr*Sx)/12)
        Slender_PhiMn = np.where(Slender, np.minimum(Slender_LTB, PhiMpx), np.nan)

        #Pre-composite Flexure Check
        PhiMn_PC_IG = np.where(~np.isnan(Compact_PhiMn), Compact_PhiMn, np.where(~np.isnan(Noncompact_PhiMn), Noncompact_PhiMn, Slender_PhiMn))

        # Pre-composite checks
        PhiVnx_IG = np.expand_dims(PhiVnx, axis = 0)
        pre_composite_V_check_IG = V_PC_IG <= PhiVnx_IG #
        pre_composite_M_check_IG = M_PC_IG <= PhiMn_PC_IG

        #Composite checks
        composite_V_check_IG = V_FL_IG <= PhiVnx_IG
        composite_M_check_IG = M_FL_IG <= Phi_Mn_IG

        # Deflecition checks
        DL_DEF_check_IG = DEF_CDL_IG <= min(Girder_Span * 12 / 360, 1)
        LL_DEF_check_IG = DEF_CLL_IG <= min(Girder_Span * 12 / 360, 1)
        TL_DEF_check_IG = DEF_CTL_IG <= Girder_Span * 12 / 240

        # Infill fitting inside girders
        infill_d = np.expand_dims(d, axis = 1)
        girder_T = np.expand_dims(T, axis = 0)
        depth_check = girder_T >= infill_d

        # Combined Check
        combined_check_IG = (pre_composite_V_check_IG & 
                             pre_composite_M_check_IG & 
                             composite_V_check_IG &
                             composite_M_check_IG &
                             DL_DEF_check_IG & 
                             LL_DEF_check_IG &
                             TL_DEF_check_IG &
                             depth_check &
                             Eco &
                             PCC_check_flat_IG
                             )
        combined_check = combined_check_II & combined_check_IG
        
        # Create a mask that selects the most economic infill of each grouping
        groups_filtered = pd.DataFrame(np.where(combined_check, Groups_array, np.nan))
        ingroup_filtered = np.where(combined_check, Ingroup_array, np.nan)
    
        mask = pd.DataFrame(False, index=groups_filtered.index, columns=groups_filtered.columns)

        for column in groups_filtered.columns:
            # Mark the first occurrence of each unique value
            first_occurrences = groups_filtered[column].duplicated(keep='first') == False
            # Set the corresponding indices in the mask to True
            mask[column][first_occurrences] = True

        final_check = combined_check & mask


        girder_sizes = np.expand_dims(shape_label, axis = 0).astype(str)
        girder_depth = np.expand_dims(d, axis = 0)
        girder_area = np.expand_dims(A, axis = 0)
        girder_Ix = np.expand_dims(Ix, axis = 0)
        girder_sw = np.expand_dims(SW, axis = 0)
    
        #--------------------------------------------------------------------
        #----------------------Vibration Calc--------------------------------
        #--------------------------------------------------------------------

        dead_load = 4   #for vibations
        damping_ratio = DR

        effective_conc_depth = selected_deck_topping_thickness + Additional_Topping

        #Deck Properties
        Ec = conc_density ** 1.5 * ms.sqrt(f_prime_c) #ksi

        #n Calc
        n = Es / (1.35 * Ec)

        #Effective Concrete Slab Width for Infills
        effective_conc_width = min(0.4 * Infill_Span * 12, infill_spacing * 12) #in

        #Transformed Concrete Slab Width for Infills
        transformed_conc_width = effective_conc_width / n #in

        #Transformed Concrete Slab Area for Infills
        transformed_conc_area = effective_conc_depth * transformed_conc_width #in^2

        #Transformed Moment of Inertia for Infills
        y_bar_infill = (transformed_conc_area * (infill_depth / 2 + Deck_Type + effective_conc_depth / 2)) / (transformed_conc_area + infill_area)

        Ij = (transformed_conc_width * effective_conc_depth ** 3 / 12 + transformed_conc_area * 
              (infill_depth / 2 + Deck_Type + effective_conc_depth / 2 - y_bar_infill) ** 2 + infill_Ix + infill_area * y_bar_infill ** 2)

        #Infill uniformly distributed load
        wj = infill_spacing*(live_load_vibrations + Final_Slab_SW + dead_load) + infill_sw

        #Infill deflection using Transformed Moment of Intertia
        DEF_j = (5 * wj * Infill_Span ** 4 * 1728) / (384 * (Es * 1000) * Ij)

        #Infill fundamental frequency
        fj = 0.18 * np.sqrt(386 / DEF_j)

        #transformed slab moment of inertia per unit width
        de = effective_conc_depth + Deck_Type / 2
        Ds = (12 * de ** 3) / (12 * n)

        #The transformed moment of inertia per unit width for Infills
        Dj = Ij / infill_spacing

        #The effective beam panel width of Infills
        Cj = 2.0
        Bj = Cj * (Ds / Dj) ** 0.25 * Infill_Span

        #is Bj <= (2/3) * floor width
        floor_width = 200 #ft
        floor_width_check = (2/3) * floor_width
        Bj_check = Bj <= floor_width_check

        #The weight of the beam panel (x1.5 because I am assuimng girder is shear connected)
        Wj = 1.5 * (wj / infill_spacing) * Bj * Infill_Span

        #Girder Transformed Moment of Inertia
        #The effective concrete slab width
        effective_conc_width_girder = min(0.2 * Girder_Span, 0.5 * Infill_Span) * 12 + min(0.2 * Girder_Span, 0.5 * Infill_Span) *12 #in

        #Transformed Concrete Slab Width for Girders
        transformed_conc_width_girder = effective_conc_width_girder / n #in

        #Transformed Concrete Slab Width of Deck
        transformed_conc_width_deck = effective_conc_width_girder / 2 / n #in

        #Transformed Concrete Slab Area for Girders
        transformed_conc_area_girder = effective_conc_depth * transformed_conc_width_girder #in^2

        #Transformed Concrete Slab Area of Deck
        transformed_conc_area_deck = Deck_Type * transformed_conc_width_deck #in^2

        # Transformed Moment of Inertia for Girders
        y_bar_girder = (transformed_conc_area_girder * (girder_depth / 2 + Deck_Type + effective_conc_depth / 2) + transformed_conc_area_deck * 
                        (girder_depth / 2 + Deck_Type / 2)) / (transformed_conc_area_girder + transformed_conc_area_deck + girder_area)

        Ig = (transformed_conc_width_girder * effective_conc_depth ** 3 / 12 + transformed_conc_area_girder * (girder_depth / 2 + Deck_Type + effective_conc_depth / 2 - y_bar_girder) ** 2 
              + transformed_conc_width_deck * Deck_Type ** 3 / 12 + transformed_conc_area_deck * (girder_depth / 2 + Deck_Type / 2 - y_bar_girder) ** 2 + girder_Ix + girder_area * y_bar_girder ** 2)
        
        #Girder uniformly distributed load
        wg = Infill_Span * (wj / infill_spacing) + girder_sw #plf

        #Girder deflection using Transformed Moment of Intertia
        DEF_g = (5 * wg * Girder_Span ** 4 *1728) / (384 * 29 * 10 ** 6 * Ig) #in

        #Infill fundamental frequency
        fg = 0.18 * np.sqrt(386 / DEF_g) #Hz

        #The transformed moment of inertia per unit width for Girders
        Dg = Ig / Infill_Span #in^4 / ft

        #The effective beam panel width of Girders
        Cg = 1.8 #Assuming infills are connected to Girder Web
        Bg = Cg * (Dj / Dg) ** 0.25 * Girder_Span #ft

        #is Bg <= (2/3) * floor length
        floor_length = 200 #ft
        floor_length_check = (2/3) * floor_length
        Bg_check = Bg <= floor_length_check

        #The weight of the girder panel
        Wg = (wg / Infill_Span) * Bg * Girder_Span #lbs

        #Floor Fundamental Frequency
        fn = 0.18 * np.sqrt(386 / (DEF_j + DEF_g)) #Hz

        #Girder Deflection Reduction
        girder_deflection_reduction = Girder_Span < Bj
        DEF_g_reduced = np.where(girder_deflection_reduction, np.maximum(((Girder_Span / Bj) * DEF_g), 0.5 * DEF_g), DEF_g)

        #equivalent panel mode panel weigh
        W = DEF_j / (DEF_j + DEF_g_reduced) * Wj + DEF_g_reduced / (DEF_j + DEF_g_reduced) * Wg

        #ratio of the peak floor acceleration to the acceleration of gravity
        ap_over_g = (65 * ms.e ** (-0.35 * fn)) / (damping_ratio * W)

        #peak acceleration
        peak_acceleration = ap_over_g * 100
        vibration_check = peak_acceleration <= tolerance_limit

        #--------------------------------------------------------------------
        #---------------------------EC Calcs---------------------------------
        #--------------------------------------------------------------------

        #Bay Area
        bay_area = Infill_Span * Girder_Span

        #Steel w_shape Weight
        steel_w_infill = infill_sw * Infill_Span * (NumInfill + 2)
        steel_w_girder = girder_sw * Girder_Span * 2
        steel_w = steel_w_infill + steel_w_girder #lbs

        #Steel deck weight
        steel_deck_w = selected_deck_SW * bay_area

        #3/4" stud weight
        if Deck_Type == 1.5:
            per_stud_w = 0.374 #lbs per stud
        elif Deck_Type == 2:
            per_stud_w = 0.437
        elif Deck_Type == 3:
            per_stud_w = 0.562
        
        studs_w_infill = per_stud_w * min_req_studs_II * (NumInfill + 2)
        studs_w_girder = per_stud_w * min_req_studs_IG * 2
        studs_w = studs_w_infill + studs_w_girder
        
        #Conc Weight
        conc_w = (selected_deck_topping_thickness + Conc_Depth_in_Flute + Additional_Topping) / 12 * conc_density * bay_area #lbs

        #Embodied Carbon
        EC = steel_w * 1.22 + steel_deck_w * 2.32 + studs_w * 1.27 + conc_w * Conc_ECV #lbCO2e

        #Embodied Carbon Scaled
        EC_scaled = EC / bay_area #lbCO2e/ft^2

        #Dictionary of variables to be used outside defenition
        results = {
            'Selected Deck': selected_deck,
            'SW_II': SW_II,
            'SW_IG': SW_IG,
            'DR': np.array(damping_ratio),
            'Peak Acceleration': peak_acceleration,
            'Embodied Carbon Scaled': EC_scaled,
            'Infill Sizes': infill_sizes,
            'Girder Sizes': girder_sizes,
            'Groups Array': Groups_array,
            'Groups': Groups,
            'Final Check': final_check,
            'W_DL_II': W_DL_II,
            'W_LL_II': W_LL_II,
            'V_FL_II': V_FL_II,
            'V_FL_IG': V_FL_IG,
            'M_FL_II': M_FL_II,
            'M_FL_IG': M_FL_IG,
            'M_PC_II': M_PC_II,
            'M_PC_IG': M_PC_IG,
            'DEF_CLL_II': DEF_CLL_II,
            'DEF_CLL_IG': DEF_CLL_IG,
            'DEF_CDL_II': DEF_CDL_II,
            'DEF_CDL_IG': DEF_CDL_IG,
            'DEF_Con_II': DEF_Con_II,
            'DEF_Con_IG': DEF_Con_IG,
            'DEF_CTL_II': DEF_CTL_II,
            'DEF_CTL_IG': DEF_CTL_IG,
            'Phi_Mn_II': Phi_Mn_II,
            'Phi_Mn_IG': Phi_Mn_IG,
            'PhiVnx_II': PhiVnx_II,
            'PhiVnx_IG': PhiVnx_IG,            
            'PhiMpx_II': PhiMpx_II,   
            'PhiMn_PC_IG': PhiMn_PC_IG,
            'Y1_II': Y1_II,
            'Y1_IG': Y1_IG,
            'Y2_II': Y2_II,
            'Y2_IG': Y2_IG,
            'camber_II': camber_II,
            'camber_IG': camber_IG,
            'I_LB_II': I_LB_II,
            'I_LB_IG': I_LB_IG,
            'Case Label II': case_label_II,
            'Case Label IG': case_label_IG,
            'effective': effective_conc_depth,
            'Total Topping': Final_Slab_Thickness,
            'Fundamental Frequency': fn,
            'Studs II': min_req_studs_II,
            'Studs IG': min_req_studs_IG,
        }

        return results

    #------------------------------------------------------------------------
    #----------------------Dataframe Creation--------------------------------
    #------------------------------------------------------------------------
    
    # Base calculation
    base_results = mitigation_method(0)

    if base_results == None:
        return None
    
    else:
        combined_matrix = np.vectorize(lambda x, y: f"Infill: {x}, Girder: {y}")(base_results['Infill Sizes'], base_results['Girder Sizes'])

    #Transforms 1d array to apply to infill values
    def expand_infill(value):
        return np.repeat(np.expand_dims(value, axis=1), repeats=273, axis=1)
    
    #Transforms 1d array to apply to girder values
    def expand_girder(value):
        value_reshape = value.reshape(1, 273)
        return np.repeat(value_reshape, 273, axis=0)

    def map_to_matrices(range_traversal_df, target_matrices_dict):
        """
        Map column and row indices from range_traversal_df to multiple target matrices.

        Parameters:
        range_traversal_df (pd.DataFrame): DataFrame containing ColIdx, RowIdx, ColumnName, and Value.
        target_matrices_dict (dict): Dictionary where keys are matrix names and values are target matrices (numpy arrays).

        Returns:
        pd.DataFrame: DataFrame with the mapped values from all target matrices as new columns.
        """
        mapped_results = range_traversal_df.copy()

        # Iterate over each target matrix in the dictionary
        for matrix_name, target_matrix in target_matrices_dict.items():
            mapped_values = []

            # Map values from the target matrix using the column and row indices
            for _, row in range_traversal_df.iterrows():
                col_idx = row["ColIdx"]
                row_idx = row["RowIdx"]

                # Retrieve the value from the matrix
                try:
                    mapped_value = target_matrix[row_idx, col_idx]
                except IndexError:
                    mapped_value = np.nan  # Handle out-of-bounds indices gracefully

                # Add the mapped value to the list
                mapped_values.append(mapped_value)

            # Add the mapped values as a new column to the DataFrame
            mapped_results[f"{matrix_name}"] = mapped_values

        return mapped_results
    
    #Dictonary of all values returned in final upsizing dataframe
    target_base_matrices = {
        "Damping Ratio": np.full((273, 273),base_results['DR']),
        "Additional Topping": np.zeros_like(base_results['Peak Acceleration']),
        "Peak Acceleration": base_results['Peak Acceleration'],
        "Embodied Carbon Scaled": base_results['Embodied Carbon Scaled'],
        "Selected Deck": np.array(base_results['Selected Deck']).flatten(),
        "Combined Sizes": combined_matrix,
        "Infill Sizes": expand_infill(base_results['Infill Sizes'].flatten()),
        "Infill Studs": expand_infill(base_results['Studs II']),
        "Infill Shear": expand_infill(base_results['V_FL_II']),
        "Infill Shear Capacity": expand_infill(base_results['PhiVnx_II']),
        "Infill PC Moment": expand_infill(base_results['M_PC_II']),       
        "Infill PC Moment Capcity": expand_infill(base_results['PhiMpx_II']), 
        "Infill Moment": expand_infill(base_results['M_FL_II']),
        "Infill Moment Capcity": expand_infill(base_results['Phi_Mn_II']),
        "Y1_II": expand_infill(base_results['Y1_II']),
        "Y2_II": expand_infill(base_results['Y2_II']),
        "Infill Contruction Load Deflection": expand_infill(base_results['DEF_Con_II']),
        "Infill Camber": expand_infill(base_results['camber_II']),
        "Infill Composite Dead Load Deflection": expand_infill(base_results['DEF_CDL_II']),
        "Infill I_LB": expand_infill(base_results['I_LB_II']),
        "Infill Composite Live Load Deflection": expand_infill(base_results['DEF_CLL_II']),
        "Infill Composite Total Load Deflection": expand_infill(base_results['DEF_CTL_II']),
        "Infill Case Label": expand_infill(base_results['Case Label II']),
        "Girder Sizes": expand_girder(base_results['Girder Sizes'].flatten()),
        "Girder Studs": expand_girder(base_results['Studs IG']),
        "Girder Shear": base_results['V_FL_IG'],
        "Girder Shear Capacity": expand_girder(base_results['PhiVnx_IG']),
        "Girder PC Moment": base_results['M_PC_IG'],       
        "Girder PC Moment Capcity": expand_girder(base_results['PhiMn_PC_IG']), 
        "Girder Moment": base_results['M_FL_IG'],
        "Girder Moment Capcity": expand_girder(base_results['Phi_Mn_IG']),
        "Y1_IG": expand_girder(base_results['Y1_IG']),
        "Y2_IG": expand_girder(base_results['Y2_IG']),
        "Girder Contruction Load Deflection": base_results['DEF_Con_IG'],
        "Girder Camber": base_results['camber_IG'],
        "Girder Composite Dead Load Deflection": base_results['DEF_CDL_IG'],
        "Girder I_LB": np.tile(np.array(base_results['I_LB_IG']).reshape(1, -1), (273, 1)),
        "Girder Composite Live Load Deflection": expand_girder(base_results['DEF_CLL_IG']),
        "Girder Composite Total Load Deflection": base_results['DEF_CTL_IG'],
        "Girder Case Label": expand_girder(base_results['Case Label IG']),
        "Total Topping": base_results['Total Topping'],
        "Fundamental Frequency": base_results['Fundamental Frequency']

    }

    def initial_values(base_results):
        """
        Designs the initial design based on structural limit states and calculates EC and peak acceleration

        Parameters:
        base_results (dict): Dictionary of all initial 

        Returns:
        pd.DataFrame: DataFrame with the initial mapped values from all target matrices as new columns.
        """

        #DataFrame of the EC Values
        EC_dataframe = pd.DataFrame(np.where(base_results['Final Check'], base_results['Embodied Carbon Scaled'], np.nan),
                                     index=base_results['Infill Sizes'].flatten(),
                                       columns=base_results['Girder Sizes'].flatten())

        # Check if EC_dataframe contains only NaN values
        if EC_dataframe.isnull().all().all():
            print(f"No valid sizes found for {percent_comp}% composite action.")
            dummy_df = pd.DataFrame({
                'ColIdx': [0],
                'RowIdx': [0]
            })
            mapped_results = map_to_matrices(dummy_df, target_base_matrices)
            # Set all values to 0
            for col in mapped_results.columns:
                mapped_results[col] = 0
            return mapped_results

        # Find the minimum value and its location
        min_location = EC_dataframe.stack().idxmin()

        # Convert row and column labels to integer indices
        row_index = EC_dataframe.index.get_loc(min_location[0])  # Convert row label to index
        col_index = EC_dataframe.columns.get_loc(min_location[1])  # Convert column label to index

        # Or alternatively, using lists:
        initial_df = pd.DataFrame({
            'ColIdx': [col_index],
            'RowIdx': [row_index]
        })


        # Map the matrices using the updated function
        mapped_results_df = map_to_matrices(initial_df, target_base_matrices)

        return mapped_results_df
    
    #Call Initial Sizing DF
    initial_values_df = initial_values(base_results)

    #-----------------------Infill Upsizing----------------------------------

    def infill_upsizing(base_results):
        """
        For intial designs expected to exceed the peak acceleration tolerance limit, infills are upsized until the limit is met

        Parameters:
        base_results (dict): Dictionary of all initial values used in design

        Returns:
        pd.DataFrame: DataFrame with the all infill upsizings mapped to values from all target matrices as new columns.
        """

        #DataFrame of the EC Values
        EC_dataframe = pd.DataFrame(np.where(base_results['Final Check'], base_results['Embodied Carbon Scaled'], np.nan),
                                     index=base_results['Infill Sizes'].flatten(),
                                       columns=base_results['Girder Sizes'].flatten())

        # Check if EC_dataframe contains only NaN values
        if EC_dataframe.isnull().all().all():
            dummy_df = pd.DataFrame(0, index=range(1), columns=list(target_base_matrices.keys()) + ['# of Upsizes'])
            return dummy_df
        
        # Find the minimum value and its location
        min_location = EC_dataframe.stack().idxmin()

        # Convert row and column labels to integer indices
        row_index = EC_dataframe.index.get_loc(min_location[0])  # Convert row label to index
        col_index = EC_dataframe.columns.get_loc(min_location[1])  # Convert column label to index

        # Create a DataFrame of the PhiMn Grouping
        Grouping_dataframe = base_results['Groups Array']
        
        #DataFrame of the PhiMn Grouping, with final check masking
        Grouping_df = pd.DataFrame(
            np.where(base_results['Final Check'], Grouping_dataframe, np.nan),
            index=base_results['Infill Sizes'].flatten(),
            columns=base_results['Girder Sizes'].flatten()
        )

        #Removes all columns to the right of the lowest observed EC value in matrix to begin Infill Upsizing
        Grouping_df= Grouping_df.iloc[:, :col_index+1]

        # Compute max and min values
        max_values = Grouping_df.max(skipna=True)
        min_values = Grouping_df.min(skipna=True)

        # Convert dictionary to list of tuples for processing
        min_values_list = list(min_values.items())

        # Initialize variables for filtering
        filtered_indices = []
        current_min = float('inf')  # Start with infinity to find the minimum

        # Traverse the list from bottom to top
        for index, item in enumerate(reversed(min_values_list)):
            column_name, value = item
            if value < current_min:
                # Only add the item if it's less than the current minimum seen
                current_min = value
                # Find the row index of this value in the respective column
                row_index = Grouping_df[column_name].tolist().index(value)
                # Add the column index, row index, column name, and value to the filtered_indices
                column_index = list(Grouping_df.columns).index(column_name)
                filtered_indices.append((len(min_values_list) - 1 - index, column_index, row_index, column_name, value))

        # Reverse the filtered_indices to restore the original order
        filtered_indices.reverse()

        # Handle the bottom column index
        bottom_column_index = filtered_indices[-1][0]
        max_value_bottom_column = max_values[bottom_column_index]

        # Create a DataFrame for sorting results
        sorting_df = pd.DataFrame(filtered_indices, columns=["Idx", "ColIdx", "RowIdx", "ColumnName", "Value"])


        # Add the bottom column max value to the sorted group
        sorted_groups = list(sorting_df["Value"]) + [max_value_bottom_column]

        # Initialize an empty list to store ranges
        ranges = []

        # Loop through the sorted_groups to create ranges
        for i in range(len(sorted_groups) - 1):
            start = int(sorted_groups[i])  # Start value of the range
            end = int(sorted_groups[i + 1])  # End value of the range
            ranges.append(list(range(start, end + 1)))  # Add range to the list

        sorting_df["Ranges"] = ranges

        # Now integrate the traverse_ranges function to process ranges
        def traverse_ranges(dataframe, sorting_df):
            """
            Traverse the dataframe for all the range values for each column specified in sorting_df.
            """
            result = []
            
            # Loop through each row in sorting_df to process its ranges
            for _, row in sorting_df.iterrows():
                col_name = row["ColumnName"]
                col_idx = row["ColIdx"]
                col_ranges = row["Ranges"]
                
                # Traverse each value in the range and find its index in the column
                for value in col_ranges:
                    if value in dataframe[col_name].values:
                        row_idx = dataframe[col_name].tolist().index(value)
                        result.append((col_idx, row_idx, col_name, value))
                    else:
                        # If value isn't present, skip it
                        continue
            
            # Convert the result into a DataFrame for easier viewing and processing
            range_traversal_df = pd.DataFrame(result, columns=["ColIdx", "RowIdx", "ColumnName", "Value"])
            return range_traversal_df

        # Call the traverse_ranges function to process ranges
        range_traversal_df = traverse_ranges(Grouping_df, sorting_df)

        # Map the matrices using the updated function
        mapped_results_df = map_to_matrices(range_traversal_df, target_base_matrices)
    
        # Reverse the rows of the DataFrame
        mapped_results_df = mapped_results_df[::-1].reset_index(drop=True)  # Reverse Rows

        # Always add the '# of Upsizes' column, regardless of peak acceleration
        mapped_results_df.insert(0, '# of Upsizes', mapped_results_df.index)
        
        # Check if there are any rows where 'Peak Acceleration' is less than 0.5
        filtered_df = mapped_results_df[mapped_results_df['Peak Acceleration'] < 0.5]

        if filtered_df.empty:
            # If no values less than 0.5, return the full dataset
            print("Infill Upsizing not possible (W36x925 reached)")
        else:
            # Find the index where 'Peak Acceleration' first drops below 0.5
            stop_index = filtered_df.index[0]
            # Slice dataframe once PA dips under 0.5
            mapped_results_df = mapped_results_df.loc[:stop_index]
        
        #Returns zeros if initial design passes for vibraitons
        if mapped_results_df['Embodied Carbon Scaled'].iloc[-1] == initial_values_df['Embodied Carbon Scaled'][0]:
            mapped_results_df[mapped_results_df.columns] = 0

        return mapped_results_df

    #-----------------------Girder Upsizing----------------------------------

    def girder_upsizing(base_results):
        """
        For intial designs expected to exceed the peak acceleration tolerance limit, girders are upsized until the limit is met

        Parameters:
        base_results (dict): Dictionary of all initial values used in design

        Returns:
        pd.DataFrame: DataFrame with the all girder upsizings mapped to values from all target matrices as new columns.
        """

        #DataFrame of the EC Values
        EC_dataframe = pd.DataFrame(np.where(base_results['Final Check'], base_results['Embodied Carbon Scaled'], np.nan),
                                    index=base_results['Infill Sizes'].flatten(),
                                    columns=base_results['Girder Sizes'].flatten())

        # Check if EC_dataframe contains only NaN values
        if EC_dataframe.isnull().all().all():
            dummy_df = pd.DataFrame(0, index=range(1), columns=list(target_base_matrices.keys()) + ['# of Upsizes'])
            return dummy_df

        # Find the minimum value and its location
        min_location = EC_dataframe.stack().idxmin()

        # Convert row and column labels to integer indices
        row_index = EC_dataframe.index.get_loc(min_location[0])  # Convert row label to index
        col_index = EC_dataframe.columns.get_loc(min_location[1])  # Convert column label to index

        # Create a DataFrame of the PhiMn Grouping
        Grouping_dataframe = base_results['Groups Array']

        #DataFrame of the PhiMn Grouping, with final check masking
        Grouping_df = pd.DataFrame(
            np.where(base_results['Final Check'], Grouping_dataframe, np.nan),
            index=base_results['Infill Sizes'].flatten(),
            columns=base_results['Girder Sizes'].flatten()
        )

        #Removes all columns to the right of the lowest observed EC value in matrix to begin Infill Upsizing
        Grouping_df = Grouping_df.iloc[:, :col_index+1]

        # Initialize a list to store the results
        max_indices = []

        # Iterate over each column to find the max value and its corresponding row
        for col_idx, column_name in enumerate(Grouping_df.columns):
            # Get the column data (drop NaN values)
            column_data = Grouping_df[column_name].dropna()
            
            if not column_data.empty:
                # Get max value and row index for this column
                max_value = column_data.max()
                row_label = column_data.idxmax()  # Row label corresponding to the max value
                row_idx = Grouping_df.index.get_loc(row_label)  # Convert label to numeric index
                max_indices.append((col_idx, row_idx, column_name, max_value))

        # Convert the list to a DataFrame
        max_indicies_df = pd.DataFrame(max_indices, columns=['ColIdx', 'RowIdx', 'ColumnName', 'MaxValue'])
        
        # Map the matrices using the updated function
        mapped_results_df = map_to_matrices(max_indicies_df, target_base_matrices)

        # Reverse the rows of the DataFrame
        mapped_results_df = mapped_results_df[::-1].reset_index(drop=True)  # Reverse Rows

        # Always add the '# of Upsizes' column, regardless of peak acceleration
        mapped_results_df.insert(0, '# of Upsizes', mapped_results_df.index)

        # Check if there are any rows where 'Peak Acceleration' is less than 0.5
        filtered_df = mapped_results_df[mapped_results_df['Peak Acceleration'] < 0.5]

        if filtered_df.empty:
            # If no values less than 0.5, return the full dataset
            print("Girder Upsizing not possible (W36x925 reached)")
        else:
            # Find the index where 'Peak Acceleration' first drops below 0.5
            stop_index = filtered_df.index[0]
            # Slice dataframe once PA dips under 0.5
            mapped_results_df = mapped_results_df.loc[:stop_index]

        #Returns zeros if initial design passes for vibrations
        if mapped_results_df['Embodied Carbon Scaled'].iloc[-1] == initial_values_df['Embodied Carbon Scaled'][0]:
            mapped_results_df[mapped_results_df.columns] = 0
        
        return mapped_results_df

    #-----------------------Additional Concrete------------------------------

    def concrete_upsizing(cap, step):
        """
        For intial designs expected to exceed the peak acceleration tolerance limit, concrete is upsized until the limit is met

        Parameters:
        cap: maximum amount of concrete that can be added
        step: interval size for how much concrete is added during each iteration

        Returns:
        pd.DataFrame: DataFrame with the all concrete upsizings mapped to values from all target matrices as new columns.
        """

        if initial_values_df is None or initial_values_df['Peak Acceleration'].iloc[0] == 0:
            dummy_df = pd.DataFrame(0, index=range(1), columns=target_base_matrices.keys())
            return dummy_df
        
        if initial_values_df['Peak Acceleration'].iloc[0] <= 0.5:
            return pd.DataFrame(0, index=range(1), columns=target_base_matrices.keys())

        # Initialize a dictionary to store the results
        all_mitigated_results = {}
        
        # Initialize a list to store peak accelerations for each iteration
        peak_accelerations = []
    
        # Initialize an empty DataFrame to collect results
        all_results_df = pd.DataFrame()

        # Check Additional_Topping
        Additional_Topping = 0

        while Additional_Topping < cap:  # Cap in inches (5 inches = 100 iterations with step 0.1)

            Additional_Topping += step  # Increment in inches
            mitigated_results = mitigation_method(Additional_Topping - step)

            if mitigated_results is None:
                break    

            # Store the mitigated results in the dictionary with the key being the Additional_Topping value
            all_mitigated_results[Additional_Topping] = mitigated_results

            EC_dataframe = pd.DataFrame(np.where(mitigated_results['Final Check'], mitigated_results['Embodied Carbon Scaled'], np.nan),
                                         index=base_results['Infill Sizes'].flatten(),
                                           columns=base_results['Girder Sizes'].flatten())

            # Check if EC_dataframe contains only NaN values
            if EC_dataframe.isnull().all().all():
                if all_results_df.isnull().all().all():
                    print("EC_dataframe contains only NaN values. Returning nothing.")
                    return None  # Exit gracefully and return nothing
                else:
                    return all_results_df
            
            # Find the minimum value and its location
            min_location = EC_dataframe.stack().idxmin()

            # Convert row and column labels to integer indices
            row_index = EC_dataframe.index.get_loc(min_location[0])  # Convert row label to index
            col_index = EC_dataframe.columns.get_loc(min_location[1])  # Convert column label to index

            combined_mitigated_matrix = np.vectorize(lambda x, y: f"Infill: {x}, Girder: {y}")(mitigated_results['Infill Sizes'], mitigated_results['Girder Sizes'])

            target_mitigated_matrices = {
            "Damping Ratio": np.full((273, 273),base_results['DR'])[row_index, col_index],
            "Additional Topping": Additional_Topping - step,
            "Peak Acceleration": mitigated_results['Peak Acceleration'][row_index, col_index],
            "Embodied Carbon Scaled": mitigated_results['Embodied Carbon Scaled'][row_index, col_index],
            "Selected Deck": np.array(mitigated_results['Selected Deck']).flatten(),
            "Combined Sizes": combined_mitigated_matrix[row_index, col_index],
            "Infill Sizes": expand_infill(mitigated_results['Infill Sizes'].flatten())[row_index, col_index],
            "Infill Studs": expand_infill(base_results['Studs II'])[row_index, col_index],
            "Infill Moment": expand_infill(mitigated_results['M_FL_II'])[row_index, col_index],
            "Infill Moment Capcity": expand_infill(mitigated_results['Phi_Mn_II'])[row_index, col_index],
            "Y1_II": expand_infill(mitigated_results['Y1_II'])[row_index, col_index],
            "Y2_II": expand_infill(mitigated_results['Y2_II'])[row_index, col_index],
            "Infill Contruction Load Deflection": expand_infill(mitigated_results['DEF_Con_II'])[row_index, col_index],
            "Infill Camber": expand_infill(mitigated_results['camber_II'])[row_index, col_index],
            "Infill Composite Dead Load Deflection": expand_infill(mitigated_results['DEF_CDL_II'])[row_index, col_index],
            "Infill I_LB": expand_infill(mitigated_results['I_LB_II'])[row_index, col_index],
            "Infill Composite Live Load Deflection": expand_infill(mitigated_results['DEF_CLL_II'])[row_index, col_index],
            "Infill Composite Total Load Deflection": expand_infill(mitigated_results['DEF_CTL_II'])[row_index, col_index],
            "Infill Case Label": expand_girder(mitigated_results['Case Label II'])[row_index, col_index],
            "Girder Sizes": expand_girder(mitigated_results['Girder Sizes'].flatten())[row_index, col_index],
            "Girder Studs": expand_girder(base_results['Studs IG'])[row_index, col_index],
            "Girder Moment": mitigated_results['M_FL_IG'][row_index, col_index],
            "Girder Moment Capcity": expand_girder(mitigated_results['Phi_Mn_IG'])[row_index, col_index],
            "Y1_IG": expand_girder(mitigated_results['Y1_IG'])[row_index, col_index],
            "Y2_IG": expand_girder(mitigated_results['Y2_IG'])[row_index, col_index],
            "Girder Contruction Load Deflection": mitigated_results['DEF_Con_IG'][row_index, col_index],
            "Girder Camber": mitigated_results['camber_IG'][row_index, col_index],
            "Girder Composite Dead Load Deflection": mitigated_results['DEF_CDL_IG'][row_index, col_index],
            "Girder I_LB": expand_girder(mitigated_results['I_LB_IG'])[col_index],
            "Girder Composite Live Load Deflection": expand_girder(mitigated_results['DEF_CLL_IG'])[row_index, col_index],
            "Girder Composite Total Load Deflection": mitigated_results['DEF_CTL_IG'][row_index, col_index],
            "Girder Case Label": expand_girder(mitigated_results['Case Label IG'])[row_index, col_index],
            "Total Topping": mitigated_results["Total Topping"],
            "Fundamental Frequency": mitigated_results['Fundamental Frequency'][row_index, col_index],
            }

            # Use row_index and col_index as a mask to pull the value for peak acceleration or any other value
            peak_acceleration = target_mitigated_matrices['Peak Acceleration']

            # Convert the target mitigated matrices dictionary to a DataFrame for this iteration
            temp_df = pd.DataFrame([target_mitigated_matrices])

            # Append the current iteration's results to the all_results_df (stacking vertically)
            all_results_df = pd.concat([all_results_df, temp_df], ignore_index=True)

            if peak_acceleration < 0.5:
                break
            
        return(all_results_df)

    # Call Upsziing Defenitions
    infill_upsizing_df = infill_upsizing(base_results)
    girder_upsizing_df = girder_upsizing(base_results)
    concrete_upsizing_df = concrete_upsizing(10, 0.25)

    # Obtain objective values for initial and mitigated designs
    initial_ec = initial_values_df['Embodied Carbon Scaled'][0]
    infill_ec = infill_upsizing_df['Embodied Carbon Scaled'].iloc[-1]
    girder_ec = girder_upsizing_df['Embodied Carbon Scaled'].iloc[-1]
    concrete_ec = concrete_upsizing_df['Embodied Carbon Scaled'].iloc[-1]

    initial_pa = initial_values_df['Peak Acceleration'][0]
    infill_pa = infill_upsizing_df['Peak Acceleration'].iloc[-1]
    girder_pa = girder_upsizing_df['Peak Acceleration'].iloc[-1]
    concrete_pa = concrete_upsizing_df['Peak Acceleration'].iloc[-1]

    # Obtain initial member values
    initial_infill = initial_values_df['Infill Sizes'][0]
    initial_infill_studs = initial_values_df['Infill Studs'][0]
    initial_infill_camber = initial_values_df['Infill Camber'][0]
    initial_girder = initial_values_df['Girder Sizes'][0]
    initial_girder_studs = initial_values_df['Girder Studs'][0]
    initial_girder_camber = initial_values_df['Girder Camber'][0]

    # Assign EC penalty if peak acceleration limit isn't met
    if infill_pa > 0.5:
        infill_ec = float('inf')
    if girder_pa > 0.5:
        girder_ec = float('inf')
    if concrete_pa > 0.5:
        concrete_ec = float('inf')

    # Assign controlling mitigation method to the 'best_method_df'
    if (initial_ec != 0 and infill_ec == 0 and girder_ec == 0 and concrete_ec == 0) and (initial_pa <= 0.5):
        condition_class = 1
        #FILL THIS 'best_methods_df' DF BELOW with NaNs
        best_method_df = initial_values_df
        print_statement = "Initial Design Works"

    elif (infill_ec != initial_ec and infill_ec != 0 and infill_ec <= min(girder_ec, concrete_ec)) and (infill_pa <= 0.5):
        condition_class = 2
        best_method_df = infill_upsizing_df
        print_statement = "Upsize Infills"

    elif (girder_ec != initial_ec and girder_ec != 0 and girder_ec <= min(infill_ec, concrete_ec)) and (girder_pa <= 0.5):
        condition_class = 3
        best_method_df = girder_upsizing_df
        print_statement = "Upsize Girders"

    elif (concrete_ec != initial_ec and concrete_ec != 0 and concrete_ec <= min(infill_ec, girder_ec)) and (concrete_pa <= 0.5):
        condition_class = 4
        best_method_df = concrete_upsizing_df
        print_statement = "Add Concrete Topping"

    else:
        condition_class = 5
        #FILL THIS 'best_methods_df' DF BELOW with NaNs
        best_method_df = initial_values_df
        print_statement = "No Possible Design"

    print(print_statement,
          '\n' + 'Initial Infill: ' + str(initial_infill) + ' ' + str(initial_infill_studs.astype(int)) + ' c= ' + str(initial_infill_camber[0]) + '\n' +
          'Initial Girder: ' + str(initial_girder) + ' [' + str(initial_girder_studs.astype(int)) + '] c= ' + str(initial_girder_camber) + '\n' +
          'Initial Deck and Topping: ' + str(base_results['Selected Deck']) + '\n' +
          'Initial Peak Acceleration: ' + str(initial_pa) + '\n' +
          'Initial Scaled EC: ' + str(initial_ec) + '\n' +
          'Mitigated Infill: ' + str(best_method_df['Infill Sizes'].iloc[-1]) + ' ' + str(best_method_df['Infill Studs'].iloc[-1].astype(int)) + ' c= ' + str(best_method_df['Infill Camber'].iloc[-1].astype(int)[0]) + '\n' +
          'Mitigated Girder: ' + str(best_method_df['Girder Sizes'].iloc[-1]) + ' [' + str(best_method_df['Girder Studs'].iloc[-1].astype(int)) + '] c= ' + str(best_method_df['Girder Camber'].iloc[-1]) + '\n' +
          'Mitigated Peak Acceleration: ' + str(best_method_df['Peak Acceleration'].iloc[-1]) + '\n' +
          'Mitigated Scaled EC: ' + str(best_method_df['Embodied Carbon Scaled'].iloc[-1])
          )

    return

sizer_and_calc(Girder_Span, Infill_Span, NumInfill, Conc_Type_num, Deck_Type, Building_Use_num, percent_comp)


Girder Upsizing not possible (W36x925 reached)
Add Concrete Topping 
Initial Infill: W12X16 [21] c= 1.25
Initial Girder: W18X40 [74] c= 0.75
Initial Deck and Topping: 1.5VL20 GA w/ 2in LWC
Initial Peak Acceleration: 1.8144274820483888
Initial Scaled EC: 17.74507435388889
Mitigated Infill: W16X26 [40] c= 1
Mitigated Girder: W24X55 [77] c= 0.75
Mitigated Peak Acceleration: 0.4626367469995589
Mitigated Scaled EC: 33.29122530944444
